In [ ]:
from __future__ import print_function
from __future__ import division

from PIL import Image

import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
# from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import random_split

import matplotlib.pyplot as plt
import itertools    # confusion matrix에서 사용

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models

In [ ]:
BASE_DIR = '../input/microsoft-rice-disease-classification-challenge'

train = pd.read_csv(os.path.join(BASE_DIR, 'Train.csv'))
print(train.shape)
train.head()

In [ ]:
train_rgb = train.loc[~train['Image_id'].str.contains('_rgn')]
train_rgb.head()

In [ ]:
test = pd.read_csv(os.path.join(BASE_DIR, 'Test.csv'))
test_rgb = test.loc[~test['Image_id'].str.contains('_rgn')]
test_rgb.head()

In [ ]:
ss = pd.read_csv(os.path.join(BASE_DIR, 'SampleSubmission.csv'))
ss.head()

In [ ]:
class Img_Dataset(Dataset):
    def __init__(self, file_path, transform, table, is_rgn=False, is_train=True):
        self.file_path = file_path
        self.transform = transform
        self.table = table
        self.is_train = is_train
        
        self.img_name_list = self.table['Image_id'].tolist()
        self.img_list = []
        if not is_rgn:    # RGB
            for img_name in self.img_name_list:
                img = Image.open(os.path.join(self.file_path, img_name))
                img_transformed = self.transform(img)
                self.img_list.append(img_transformed)
        else:
            for img_name in self.img_name_list:
                img_rgn = Image.open(os.path.join(self.file_path, img_name.replace('.jpg', '_rgn.jpg')))
                img_rgn_transformed = self.transform(img_rgn)
                self.img_list.append(img_rgn_transformed)
        
        if self.is_train:
            self.label_list = [0 if label == 'blast' else (1 if label == 'brown' else 2) for label in self.table['Label'].tolist()]
  
    def __len__(self):
        return len(self.img_name_list)
  
    def __getitem__(self, index):
        if self.is_train:
            return self.img_list[index], self.label_list[index]
        else:
            return self.img_list[index]

In [ ]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def make_data_loader(batch_size=128, split=0.8):
    IMG_DIR = '../input/microsoft-rice-disease-classification-challenge/Images'
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    # train data
    train_dataset = Img_Dataset(IMG_DIR, transform, train_rgb)

    train_size = int(len(train_dataset) * split)
    val_size = len(train_dataset) - train_size

    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2)
    # whole_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    # test data
    test_dataset = Img_Dataset(IMG_DIR, transform, test_rgb, is_train=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    return train_loader, val_loader, test_loader

In [ ]:
def train_model(device, model, train_loader, val_loader, criterion, optimizer, num_epochs=5, early_stopping=None):
    model = model.to(device)

    dl = {'train': train_loader,
          'val': val_loader}
    
    val_label = []
    val_pred = []

    val_loss = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dl[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients -> backward시 필요
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    # print(preds)
                    # print(labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    if phase == 'val' and epoch == num_epochs - 1:
                        val_label += labels.tolist()
                        val_pred += preds.tolist()

                # statistics
                running_loss += loss.item() * inputs.size(0)  # size(0) : batch size (첫 번째 차원 개수)
                                                              # item() : tensor에서 저장된 값만 가져오기
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dl[phase].dataset)  # 이렇게 나누면 epoch당 평균 loss가 됨
            epoch_acc = running_corrects.double() / len(dl[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            val_loss = epoch_loss

        if early_stopping != None:
            early_stopping(val_loss, model) # epoch_loss에는 validation loss가 저장
            if early_stopping.early_stop:
                print("Early stopping")
                break

    # load best model weights
    model.load_state_dict(torch.load(early_stopping.path))
    return model, val_label, val_pred

In [ ]:
# confusion matrix 시각화
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, target_names=None, labels=True):
    accuracy = np.trace(cm) / float(np.sum(cm))

    cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(9, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    thresh = cm.max() / 2

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)

    if labels:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, "{:,}".format(cm[i, j]), horizontalalignment="center",
                     color="white" if cm[i,j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
def test_model(device, model, test_loader):
    test_pred = []

    model.eval()
    model = model.to(device)
    with torch.set_grad_enabled(False):
        for features in test_loader:
            features = features.to(device)

            outputs = model(features.to(torch.float))
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            test_pred.append(probabilities.tolist())

    return test_pred

# DeiT

In [ ]:
!pip install timm requests

In [ ]:
class myDeiT(nn.Module):
    def __init__(self, num_classes):
        super(myDeiT, self).__init__()
        self.model_ft = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
        # set_parameter_requires_grad(self.model_ft)
        # self.first_conv = nn.Conv2d(6, 3, kernel_size=1)
        
        num_ftrs = self.model_ft.head.in_features
        self.model_ft.head = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # x = self.first_conv(x)
        out = self.model_ft(x)
        return out

In [ ]:
deit_tf = myDeiT(3)

In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader(batch_size=50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(deit_tf.parameters(), lr=3e-4)
# optimizer = optim.SGD(resnet_ft4.parameters(), lr=3e-4, momentum=0.9)
criterion = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=3, verbose=True, delta=0.00001, path="deit2_checkpoint.pt")

In [ ]:
deit_tf, val_label, val_pred = train_model(device, deit_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

In [ ]:
# inference
deit_tf.load_state_dict(torch.load(os.path.join('deit2_checkpoint.pt')))

In [ ]:
test_pred = test_model(device, deit_tf, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_deit_3.csv", index=False)
ss.head()

# Use SqueezeNet

In [ ]:
class mySqueezeNet(nn.Module):
    def __init__(self, num_classes):
        super(mySqueezeNet, self).__init__()
        self.model_ft = models.squeezenet1_0(pretrained=True)
        # set_parameter_requires_grad(self.model_ft)
        # self.first_conv = nn.Conv2d(6, 3, kernel_size=1)
        
        self.model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        self.model_ft.num_classes = num_classes

    def forward(self, x):
        # x = self.first_conv(x)
        out = self.model_ft(x)
        return out

In [ ]:
squeeze_tf = mySqueezeNet(3)

In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader(batch_size=128)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(squeeze_tf.parameters(), lr=3e-4)
# optimizer = optim.SGD(resnet_ft4.parameters(), lr=3e-4, momentum=0.9)
criterion = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=3, verbose=True, delta=0.00001, path="deit2_checkpoint.pt")

In [ ]:
squeeze_tf, val_label, val_pred = train_model(device, squeeze_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

In [ ]:
test_pred = test_model(device, squeeze_tf, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_squeeze.csv", index=False)
ss.head()

# EfficientNet

In [ ]:
ef = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)

In [ ]:
print(ef)

In [ ]:
class myEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super(myEfficientNet, self).__init__()
        self.model_ft = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_widese_b4', pretrained=True)
        # set_parameter_requires_grad(self.model_ft)
        # self.first_conv = nn.Conv2d(6, 3, kernel_size=1)
        
        num_ftrs = self.model_ft.classifier.fc.in_features
        self.model_ft.classifier.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # x = self.first_conv(x)
        out = self.model_ft(x)
        return out

In [ ]:
efficient_tf5 = myEfficientNet(3)

In [ ]:
print(efficient_tf)

In [ ]:
# train
# train_loader, val_loader, test_loader = make_data_loader(batch_size=50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# optimizer = optim.Adam(efficient_tf4.parameters(), lr=3e-4)
optimizer = optim.SGD(efficient_tf5.parameters(), lr=1e-3, momentum=0.9)
criterion = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=3, verbose=True, delta=0.00001, path="eff_checkpoint.pt")

In [ ]:
efficient_tf5, val_label, val_pred = train_model(device, efficient_tf5, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

In [ ]:
test_pred = test_model(device, efficient_tf5, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_efficient.csv", index=False)
ss.head()

# ResNeXt50

In [ ]:
class myResNeXt(nn.Module):
    def __init__(self, num_classes):
        super(myResNeXt, self).__init__()
        self.model_ft = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
        # set_parameter_requires_grad(self.model_ft)
        # self.first_conv = nn.Conv2d(6, 3, kernel_size=1)
        
        num_ftrs = self.model_ft.fc.in_features
        self.model_ft.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # x = self.first_conv(x)
        out = self.model_ft(x)
        return out

In [ ]:
resnext = myResNeXt(3)

In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader(batch_size=50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# optimizer = optim.Adam(efficient_tf4.parameters(), lr=3e-4)
optimizer = optim.SGD(resnext.parameters(), lr=3e-4, momentum=0.9)
criterion = nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=3, verbose=True, delta=0.00001, path="eff_checkpoint.pt")

In [ ]:
resnext, val_label, val_pred = train_model(device, resnext, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

In [ ]:
test_pred = test_model(device, resnext, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_resnext.csv", index=False)
ss.head()